In [1]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow import keras
#from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
#from keras.optimizers import Adam
#from Data_Preparation import *
#from keras.utils import to_categorical
from scipy.spatial import distance_matrix
import os
import cv2
import numpy as np
import SimpleITK as sitk
import glob
import os
from scipy.ndimage import distance_transform_cdt
import pickle
from matplotlib import pyplot as plt
#import matplotlib.image as mpimg
#import imutils
%matplotlib inline

## Loading the  Model

In [4]:

model = keras.models.load_model('//ibs9010/current_data/Data_Keshav/image/Classification_model/NeuronClassifierModels/BinaryClassifier_proper_bigtile_50epoch_trial5data.h5')

## Running Inference

In [ ]:
img_size = 70 # fix the size of the image
#Path = "//ibs9010/current_data/Data_Keshav/image/Classification_model/Data/Newtest"
Path = "//ibs9010/current_data/Data_Keshav/Cropped_Images/S1/"
#Path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/testingcrop/S051/'
Testing_image = []# list of testing images
image_name = []# list of image names

for img in os.listdir(Path): #gives the images from the path
    img_array = cv2.imread(os.path.join(Path,img),cv2.IMREAD_GRAYSCALE) # read every image in gray scale from the given path
    Testing_image.append(img_array)
    image_name.append(img)
    
print("Number of test images:",len(Testing_image))
#Running Inference
Test_image = np.array(Testing_image).reshape(-1,img_size,img_size,1)
#make an array of every element of list from Testing_image and then reshape them
Test_image = Test_image/255.0
scores = model.predict_classes(Test_image)

# Results
neuron_image_counter = bkgd_image_counter = 0
for i in range(len(Test_image)):
    #print("Image:",image_name[i],"with score:",scores[i]) 
    if scores[i]==0:
        bkgd_image_counter +=1
    if scores[i]==1:
        neuron_image_counter += 1
    
print("Total testing images:",len(Testing_image))
print("bkgd_image_counter:",bkgd_image_counter,"neuron_image_counter:",neuron_image_counter)


## Saving classified tiles into respective folders

dest_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles'
#dest_path = "//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/NewTestsingleneurontiles"
directory = '//ibs9010/current_data/Data_Keshav/image/Classification_model/Data/Test/'

# Delete any exiting images before saving
import glob
os.chdir(dest_path)
files=glob.glob('*.tif')
for filename in files:
    os.unlink(filename)
    
#need the images of class1 to be appended in a separate list
single_image_list  = [] # contains list of filenames of single neurons
#sn_images = [] # conatins tiles of single neurons #(actual images itself)
for k in range(1,len(Test_image)):
    if scores[k]==1:
        #print(Test_image[i])
        #print("Image ",image_name[k])
        #sn_images.append(Test_image[i])
        single_image_list.append(image_name[k])
        
#print(len(single_image_list))  
#image_name[i]
for i in range(len(Test_image)):
    if scores[i]==1:
        #print("Image:",image_name[i])
        #print("TestImage:",Test_image[i])
        #cv2.imwrite(image_name[i], Test_image[i])
        cv2.imwrite(os.path.join(dest_path,image_name[i]), Test_image[i]*255)
        #pass
        
# Generating pickle file For Single neurons
import pickle
with open('//ibs9010/current_data/Data_Keshav/image/Classification_model/1july_binary_SN_MG48_3day_bs_S051.pkl', 'wb') as f:
    pickle.dump(single_image_list, f)        

In [9]:
'''
#dest_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/NewTestbackgroundneurontiles'
dest_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/backgroundneurontiles'
directory = '//ibs9010/current_data/Data_Keshav/image/Classification_model/Data/Test/'

# Delete all exiting images
import glob
os.chdir(dest_path)
files=glob.glob('*.tif')
for filename in files:
    os.unlink(filename)

#need the images of class1 to be appended in a separate list
bkgd_image_list  = [] # contains list of filenames of single neurons
#sn_images = [] # conatins tiles of single neurons #(actual images itself)
for k in range(1,len(Test_image)):
    if scores[k]==0:
        #print(Test_image[i])
        #print("Image ",image_name[k])
        #sn_images.append(Test_image[i])
        bkgd_image_list.append(image_name[k])
        
print(len(bkgd_image_list))  
#image_name[i]
for i in range(len(Test_image)):
    if scores[i]==0:
        #print("Image:",image_name[i])
        #print("TestImage:",Test_image[i])
        #cv2.imwrite(image_name[i], Test_image[i])
        cv2.imwrite(os.path.join(dest_path,image_name[i]), Test_image[i]*255)
        #pass
'''        

282


# Landmarks Generator

In [ ]:
# Function to determine local coordinates in a tile that has been classified as to having containing Neurons
def local_coordinates(file):
    x_coordinates = [] #x-coordinate
    y_coordinates = [] #y-coordinate
    z_coordinates = [] #y-coordinate
    coordinates = [] #x,y-coordinate put together
    #print(file)
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # global thresholding to create binarized image
    ret1,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    # Otsu's thresholding
    ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    #plt.imshow(th2,cmap = "gray")
    #plt.show()
    
    #Gaussian blurring to smoothen the image
    #blur_image = cv2.GaussianBlur(th2,(5,5),0)
    blur_image = cv2.medianBlur(th2,5)
    #plt.imshow(blur_image,cmap = "gray")
    #plt.show()
    
    #image array
    #npimage = np.array(th2)
    npimage = np.array(blur_image)
    
    indsx,indsy = np.where(npimage>0)
    x,y = [indsy.mean(),indsx.mean()]
    #print(x,y)
    Cx,Cy = int(x), int(y)
    
    cv2.circle(img, (Cx, Cy), 1, (255, 0, 0), 1)
   
    # show the image
    #plt.imshow(img,cmap = "gray")
    #plt.show()
    
    return x,y

# Path to obatin the classified tiles
path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/*'
output_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/landmarks/'

# Pickle files in order to obtain (from the Cropping function to get) Vertices and tile counter data and also list of neuron tiles after classification
with open('//ibs9010/current_data/Data_Keshav/image/Classification_model/11aug_trial5_binary_SN_MG48_3day_bs_S33.pkl','rb') as f:
    loaded_obj = pickle.load(f)

with open('//ibs9010/current_data/Data_Keshav/image/Classification_model/global_xy_MG48_3day_bs_S33_bigtile_piamasked.pkl','rb') as g:
    loaded_data = pickle.load(g)#cropped_data

#dest_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/singleneurontiles/'
dest_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/'
#dest_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/ThreeClasses/singleneurontiles/'
global_list = []
coordinates = []
for item in loaded_obj:
    local_x, local_y = local_coordinates(dest_path+item)
    filename= item[:-4]
    #print(filename)
    
    num = loaded_data[loaded_data['Tile_Counter'] == int(filename)].index[0]
    #num = loaded_data[loaded_data['Correct_Tile_Counter'] == int(filename) ].index[0]
    
    global_list = loaded_data.at[num, 'Vertices']
    #print(num,global_list)
    global_x = global_list[3]+  local_x
    global_y = global_list[0] + local_y
    globals = [global_x,global_y]
    coordinates.append(globals)
    
# Merging landmarks for partials!

from math import sqrt
templist =[]
newcoordinatelist = []

for index_p1,p1 in enumerate(coordinates):
    for p2 in coordinates[index_p1+1:]:
        x2 = p2[0]
        x1 = p1[0]
        y2 = p2[1]
        y1 = p1[1]
        #print(x2,x1)
        #print(y2,y1)
        dist = sqrt((x2 - x1)** 2.0 + (y2 - y1)**2.0)
        
        if dist <20:
            print(dist)
            templist.append(p1)
            templist.append(p2)
            avg = [(p1[0]+ p2[0])/2, (p1[1]+ p2[1])/2]
            print(avg)
            newcoordinatelist.append(avg)

for p in coordinates:
    if p not in templist:
        newcoordinatelist.append(p)

#adding zero for z dimension for landmark files:
for item in newcoordinatelist:
    item.append(0.5)

modified_coordinates= []
for new in newcoordinatelist:
    x_modified = new[0]*0.868
    y_modified = new[1]*0.868
    z_modified = new[2]*0.868
    modified_coordinates.append([x_modified,y_modified,z_modified])

# Function for Creating LandmarkcAscii Files
def writeLandmarkCoords(filename, landmarkList):
  with open(filename, 'w') as file:
     file.write("# Avizo 3D ASCII 2.0 \n")
     file.write("define Markers ")
     file.write("{}".format(len(landmarkList)))
     file.write("\n")
     file.write("Parameters {\n")
     file.write("    NumSets 1,\n")
     file.write("    ContentType \"LandmarkSet\"\n")
     file.write("}\n")

     file.write("Markers { float[3] Coordinates } @1\n")

     file.write("# Data section follows\n")
     file.write("@1\n")
     for landmark in landmarkList:
         file.write("{}".format(landmark[0]))
         file.write(" ")
         file.write("{}".format(landmark[1]))
         file.write(" ")
         file.write("{}".format(landmark[2]))
         file.write("\n")
        
#Calling function to create respective landmark file
writeLandmarkCoords(output_path+"11aug_trial5_MG48_binary_S33.landmarkAscii", modified_coordinates)

# TP, FP, FN Calculator

In [ ]:
def read_landmarks(filename,only_2d=True):
    manual_landmark_list = []
    lines = []
    with open(filename, 'r') as csb:
        lines = csb.readlines()
    number = 0

    # for each manual landmark
    for line in lines:
        #print line
        if line.startswith("@1"):
            number = 1
            continue
        if number == 1 and line.isspace() == False:
            #print line.split()
            pt = list(map(float,line.split()))
            #pt = [pt[0]*self.axis_directions[0],pt[1]*self.axis_directions[1],pt[2]*self.axis_directions[2]]
            manual_landmark_list.append([pt[0],pt[1]])

    return manual_landmark_list

def write_landmarks(landmarks,filename):
    
    with open(filename, 'w') as file:
        file.write("# Avizo 3D ASCII 2.0 \n")
        file.write("define Markers ")
        file.write("{}".format(len(landmarks)))
        file.write("\n")
        file.write("Parameters {\n")
        file.write("    NumSets 1,\n")
        file.write("    ContentType \"LandmarkSet\"\n")
        file.write("}\n")

        file.write("Markers { float[3] Coordinates } @1\n")

        file.write("# Data section follows\n")
        file.write("@1\n")

        if len(landmarks)!=0:
            for landmark in landmarks:
                ##print(landmark[0][0])
                file.write("{}".format(landmark[0]))
                file.write(" ")
                file.write("{}".format(landmark[1]))
                file.write(" ")
                if len(landmark) > 2:
                    file.write("{}".format(landmark[2]))
                else:
                    file.write("{}".format(0))
                file.write("\n")

#manual_landmarks_file = '/nas1/Data_aman/00_Rabies/MG48_3Day_bs/01_Data/00_Landmark/000_manual/00_original/S051_Landmarks.landmarkAscii'
#auto_landmarks_file_full = '/nas1/Data_Mythreya/RabiesCountingProject/22june_MG48_S51_trinary.landmarkAscii' 
manual_landmarks_file = '//ibs9010/current_data/Data_Keshav/image/Classification_model/manual_landmark_files/S033_Landmarks.landmarkAscii'
auto_landmarks_file_full="//ibs9010/current_data/Data_Keshav/image/Classification_model/landmarks/11aug_trial5_MG48_binary_S33.landmarkAscii"


#auto_landmarks_file_partial = '/nas1/Data_Mythreya/RabiesCountingProject/MG48_S51-26may-PN.landmarkAscii'
error_resolution = 25 # radius around manual landmark where we compare , in microns
output_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/LMfiles/'
exp = 'S020'

manual_landmarks = read_landmarks(manual_landmarks_file)
auto_landmarks_full = read_landmarks(auto_landmarks_file_full)
#auto_landmarks_partial = read_landmarks(auto_landmarks_file_partial)

auto_landmarks_all = []
for pt in auto_landmarks_full:
    auto_landmarks_all.append(pt)
# for pt in auto_landmarks_partial:
#     auto_landmarks_all.append(pt)
    
manual_np = np.array(manual_landmarks)
auto_np = np.array(auto_landmarks_all)


confusion_mat = (distance_matrix(manual_np,auto_np) < error_resolution)


TP = (confusion_mat.sum(axis=1)>0).sum()
FN = (confusion_mat.sum(axis=1)==0).sum()
FP = (confusion_mat.sum(axis=0)==0).sum()

TP_per = (TP/len(manual_landmarks))*100
FN_per = (FN/len(manual_landmarks))*100
FP_per = (FP/len(manual_landmarks))*100

TP_locations = np.where(confusion_mat.sum(axis=1))
FN_locations = np.where(confusion_mat.sum(axis=1)==0)
FP_locations = np.where(confusion_mat.sum(axis=0)==0)

write_landmarks(manual_np[TP_locations],output_path+exp+'_Matches(TP).landmarksAscii')
write_landmarks(manual_np[FN_locations],output_path+exp+'_Misses(FN).landmarksAscii')
write_landmarks(auto_np[FP_locations],output_path+exp+'_FP.landmarksAscii')


print("TP,FN,FP:",TP,FN,FP)
print("TP_per,FN_per,FP_per:",TP_per,FN_per,FP_per)